In [11]:
import pandas as pd
import dotenv  
import os

dotenv.load_dotenv()

True

In [12]:
req_diarie_env = os.getenv("REQ_DIARIE_CSV")
st_diarie_env = os.getenv("ST_DIARIE_CSV")

print(req_diarie_env)
print(st_diarie_env)


../src/GE-data-swe/GE_Krav_Diarie.csv
../src/GE-data-swe/GE_STs_Diarie_v2.csv


In [16]:
req_df = pd.read_csv(req_diarie_env)
req_headers = req_df.columns.tolist()
print(req_headers)

['Nivå 1', 'Nivå 2', 'Nivå 3', 'GE KravID', 'Rubrik', 'Beskrivning', 'Prioritet', 'Status', 'Acceptanskriterier', 'ID', 'Arkiverad']


In [36]:
st_df = pd.read_csv(st_diarie_env)
st_headers = st_df.columns.tolist()
print(st_headers)
print(st_df.head(1))

['ID', 'Rubrik', 'Info & Förberedelser', 'Kravområde', 'Länkade krav', 'Beskrivning', 'Beskrivning.1', 'Beskrivning.2', 'Beskrivning.3', 'Beskrivning.4', 'Beskrivning.5', 'Beskrivning.6', 'Beskrivning.7', 'Beskrivning.8', 'Beskrivning.9', 'Beskrivning.10', 'Beskrivning.11', 'Beskrivning.12', 'Beskrivning.13', 'Beskrivning.14', 'Beskrivning.15', 'Beskrivning.16', 'Beskrivning.17', 'Beskrivning.18', 'Beskrivning.19', 'Beskrivning.20', 'Beskrivning.21', 'Beskrivning.22', 'Beskrivning.23', 'Beskrivning.24']
   ID                                         Rubrik  \
0   1  Konvertering och Filformat hanterade i Diarie   

                                Info & Förberedelser  Kravområde  \
0  Intention: Verifiera att kravställda filformat...  Funktioner   

                                        Länkade krav  \
0  5 - Filformatsstöd| 6 - Originalfiler| 17 - Au...   

                                         Beskrivning  \
0  Roll=handläggare.\nI befintligt diarieärende s...   

               

In [38]:
import re
def merge_columns_with_regex(df):
    columns_with_beskrivning = [col for col in df.columns if re.search(r'Beskrivning', col)]
    
    if not columns_with_beskrivning:
        raise ValueError("No columns containing 'Beskrivning' found.")
    
    start_column = df.columns.get_loc(columns_with_beskrivning[0])  # Get the index of the first column
    end_column = df.columns.get_loc(columns_with_beskrivning[-1]) + 1  # Get the index after the last column
    print(start_column)
    # Select the columns that need to be merged
    columns_to_merge = df.columns[start_column:end_column]

    df.iloc[:,start_column ] = df[columns_to_merge].apply(
        lambda row: ' '.join(
            # Add a period to the end of the value if it does not already end with a period
            f"{value}." if not value.endswith('.') else value
            # Iterate over the non-empty cells in the row and convert the values to strings
            # (to avoid issues with NaN values)
            for value in row.dropna().astype(str)
        ),
        # Apply the lambda function to each row in the DataFrame
        # axis=1 specifies that the lambda function should be applied to each row, not each column
        axis=1
    )

    # Drop the now unnecessary columns, except for the column at index 2 ('C')
    df.drop(columns=columns_to_merge[1:], inplace=True)

    return df

df = merge_columns_with_regex(st_df)
print(df.head()['Beskrivning'])
#df.to_csv("st_diarie.csv", index=False)


5
0    Roll=handläggare.\nI befintligt diarieärende s...
1    Roll=huvudregistrator\nSkapa ett nytt diarieär...
2    Välj ett befintligt skapat ärende. Lägg till b...
3    Roll=huvudregistrator. På startsidan klicka på...
4    Lägg märke till utseendet på skrivbordet. På s...
Name: Beskrivning, dtype: object
